In [1]:
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import scipy as sc
import csv
import plotly as py
import googlemaps as goog
import geopy as geo
import sklearn as skl
from sklearn import cluster

# import data table as a data frame
MedicareTable = pd.read_table('D:\Medicare_Aggregate\Medicare_Physician_and_Other_Supplier_NPI_Aggregate_CY2014.txt', sep='\t',engine='python')

In [3]:
#Display 100 rows from columns
#MedicareTable.loc[1:20,:]

#pd.describe(MedicareTable.loc[:, 'average_Medicare_standard_amt'])
#MedicareTable['total_drug_unique_benes'].dropna().describe()

In [4]:
#submittedCharge = MedicareTable[MedicareTable['nppes_entity_code']=='I'].loc[:,'total_submitted_chrg_amt']
#medicarePayment = MedicareTable[MedicareTable['nppes_entity_code']=='I'].loc[:,'total_medicare_payment_amt']
#
#plt.scatter(submittedCharge, (medicarePayment/submittedCharge))
#plt.show()

In [5]:
# Create new matrix, selectedVarMatrix, containing only our three variables to cluster on
selectedVar = ['total_services','total_submitted_chrg_amt','total_medicare_stnd_amt']
reducedVarMatrix = MedicareTable[MedicareTable['nppes_entity_code']=='I'].loc[:,selectedVar]
#figure = pd.tools.plotting.scatter_matrix(selectedVarMatrix, alpha=0.2, figsize=(15,15))
#plt.show()


In [6]:
# Create reducedVarMatrix, which eliminates any tuples containing 
#reducedVarMatrix = selectedVarMatrix[pd.notnull(MedicareTable['total_services'])].loc[:,:]
#reducedVarMatrix = reducedVarMatrix[pd.notnull(reducedVarMatrix['total_submitted_chrg_amt'])].loc[:,:]
#reducedVarMatrix = reducedVarMatrix[pd.notnull(reducedVarMatrix['total_medicare_stnd_amt'])].loc[:,:]
#reducedVarMatrix = selectedVarMatrix

In [7]:
# Normalize variables
normVarMatrix = skl.preprocessing.normalize(reducedVarMatrix, norm='l1', axis=0)

scale = skl.preprocessing.StandardScaler().fit(reducedVarMatrix[:])
normVarMatrix = scale.transform(reducedVarMatrix[:])

print("Column one mean is now: ", np.mean(normVarMatrix[:,0]))
print("Column two standard deviation is now: ", np.std(normVarMatrix[:,1]))

Column one mean is now:  4.58262581174e-17
Column two standard deviation is now:  1.0


In [8]:
#Split data into 5 random samples
#randomSampleLength = len(normVarMatrix[:,0])
#random = np.random.uniform(low = 0.0, high=5.0, size=randomSampleLength)

#random1 = all(0.0 <= random < 1)
#sample1 = normVarMatrix[(0.0 <= random < 1), :]
#random >= 0

#sample1 = normVarMatrix[0:50000,:]

In [9]:
kmeans = cluster.KMeans(n_clusters = 5)
kmeans.fit(normVarMatrix)

centroid = kmeans.cluster_centers_
labels = kmeans.labels_

print(centroid)
print(labels)

[[ -0.11783874  -0.25622414  -0.24232071]
 [ 27.68840978  16.01640696  17.64281585]
 [  0.19939492   0.75140139   0.68069987]
 [  7.69666604   8.88652929   9.44396467]
 [  1.49311371   3.45515631   3.23427449]]
[2 2 0 ..., 0 0 0]


In [68]:
color_map = {0:'r',
            1:'b',
            2:'c',
            3:'y',
            4:'k',
            5:'m',
            6:'y',
            7:'w',
            8:'b',
            9:'y', }

label_color = [color_map[l] for l in labels]

plt.scatter(reducedVarMatrix.loc[:,'total_submitted_chrg_amt'],reducedVarMatrix.loc[:,'total_medicare_stnd_amt'],c=label_color)
plt.show()

In [11]:
#figure = pd.tools.plotting.scatter_matrix(selectedVarMatrix, alpha=0.2, figsize=(15,15))
#plt.show()

In [12]:
zipTable=MedicareTable[MedicareTable['nppes_entity_code']=='I']
outliers=zipTable[labels==1].loc[:,'nppes_provider_zip']
outliers=np.random.choice(outliers,size=300,replace=False)

In [13]:
newZips = np.arange(len(outliers))

for i in range(len(outliers)):
    if int(outliers[i]) > 99999:
        remainder = int(outliers[i]) % 10000
        newZips[i] = int(outliers[i]) - remainder
        newZips[i] = int(outliers[i])/10000
    else:
        remainder = int(outliers[i]) % 10000000
        newZips[i] = int(outliers[i]) - remainder
        newZips[i] = int(outliers[i])/10000

In [14]:
for i in range(len(newZips)):
    if newZips[i] < 10000:
        np.delete(newZips, i)

In [15]:
longitudes = np.arange(len(outliers))
latitudes = np.arange(len(outliers))
geolocator = geo.geocoders.Nominatim()

longitudes = longitudes.astype(float)
latitudes = latitudes.astype(float)

In [24]:
for i in range(len(outliers)):
    zipcode = str(newZips[i])
    USA = ", United States"
    search = zipcode + USA
    location = geolocator.geocode(search)
    if ((location is None) == False):
        latitudes[i] = location.latitude
        longitudes[i] = location.longitude

In [56]:
to_delete = np.arange(len(longitudes))

for i in np.arange(len(longitudes)):
    if (-125 <= longitudes[i] <= -60) == False:
        to_delete[i] = -1

for i in np.arange(len(longitudes)-1, -1, -1):
    if to_delete[i] == -1:
        longitudes = np.delete(longitudes,i)
        latitudes = np.delete(latitudes,i)

In [65]:
# Normalize variables
coordinates = np.hstack((longitudes.reshape(-1,1),latitudes.reshape(-1,1)))

normCoordMatrix = skl.preprocessing.normalize(coordinates, norm='l1', axis=0)

scale = skl.preprocessing.StandardScaler().fit(coordinates[:])
normCoordMatrix = scale.transform(coordinates[:])

print("Column one mean is now: ", np.mean(normVarMatrix[:,0]))
print("Column two standard deviation is now: ", np.std(normVarMatrix[:,1]))

Column one mean is now:  4.58262581174e-17
Column two standard deviation is now:  1.0


In [66]:
kmeans = cluster.KMeans(n_clusters = 4)
kmeans.fit(normCoordMatrix)

centroid = kmeans.cluster_centers_
labels = kmeans.labels_

print(centroid)
print(labels)

[[ 0.59957129  1.06120171]
 [-1.93225266  0.37244527]
 [ 0.56909128 -1.35035473]
 [ 0.02511423 -0.10914468]]
[2 0 2 2 3 1 0 2 3 0 2 0 2 0 2 2 1 0 1 2 1 1 3 0 3 2 2 2 0 2 0 1 3 1 0 0 1
 2 1 0 0 0 3 0 0 0 3 3 0 0 1 3 3 1 3 3 0 3 2 0 0 0 2 2 3 1 1 0 3 2 0 2 1 3
 3 1 3 0 0 2 3 1 0 3 3 0 0 2 3 3 2 3 2 0 0 0 3 2 0 3 3 3 1 2 0 1 3 2 2 0 3
 2 1 2 1 0 3 1 3 1 0 0 0 3 3 2 0 1 1 1 2 0 1 3 2 2 1 1 1 2 3 0 2 2 1 3 0 2
 2 2 2 3 3 2 0 0 2 0 2 0 1 1 2 3 0 3 2 3 2 1 0 3 2 1 0 3 1 3 0 3 3 2 0 2 2
 0 1 2 3 3 2 1 3 2 3 0 0 3 1 3 1 3 2 2 1 0 2 3 2 2 0 3 3 0 3 0 2 0 2 3 0 1
 3 3 2 3 0 3 3 2 3 3 0 3 2 3 3 3 2 1 0 2 0 2 1 3 0 2 0 3 0 0 3 0 1 0 0 0 2
 0 1 0 3 0 0 3 0 3 3 2 0 2 1 2 3 0 0 3 2 1 3 0 1]


In [64]:
print(coordinates)

[[ -82.37184447   29.67751658]
 [ -87.34128389   41.41434988]
 [ -81.92097038   28.92674843]
 [ -81.8021341    26.1233055 ]
 [ -96.6079765    39.1897624 ]
 [-119.75834381   36.86679307]
 [ -88.91143476   38.32357607]
 [ -82.7261977    28.02935728]
 [ -92.3430622    34.75162206]
 [ -76.08085951   38.77357043]
 [ -81.8021341    26.1233055 ]
 [ -78.06631614   36.53935095]
 [ -81.7451949    26.662534  ]
 [ -88.329984     42.03710903]
 [ -82.6759185    28.20991524]
 [ -81.7165369    26.252447  ]
 [-122.91646147   39.05140884]
 [ -73.78572279   40.91691586]
 [-119.75834381   36.86679307]
 [ -81.9235574    26.5387009 ]
 [-122.2891414    37.8908683 ]
 [-112.05897023   33.5080889 ]
 [ -92.3430622    34.75162206]
 [ -73.1423127    40.9123844 ]
 [ -82.6293824    34.53104309]
 [ -80.0603851    26.70986601]
 [ -80.2967122    27.29098222]
 [ -99.49205921   27.54442891]
 [ -68.80120689   44.79615665]
 [ -82.62850315   27.49154138]
 [ -72.65099006   40.93178694]
 [-115.13250519   36.12861041]
 [ -97.6

In [ ]:
label_color = [color_map[l] for l in labels]
plt.scatter(coordinates[:,0],coordinates[:,1],c=label_color)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Medicare Audit Regions")
plt.show()